# Introduction

First submission :

<a href='#1'>1. Loading Data</a>

<a href='#2'>2. Combining Datasets</a>

<a href='#3'>3. Missing Values</a>

<a href='#4'>4. Model Training</a>

<a href='#5'>5. Model Predictions</a>

# <a id='1'>1. Loading Data</a>

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import sklearn as sk
import missingno as msno 
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [ ]:
#load data
train = pd.read_csv('/kaggle/input/ashrae-energy-prediction/train.csv')
train['timestamp'] = pd.to_datetime(train['timestamp']) #the train dataset contains a 'timestamp' column we convert to a datetime object for ease of use
test = pd.read_csv('/kaggle/input/ashrae-energy-prediction/test.csv')
test['timestamp'] = pd.to_datetime(test['timestamp'])
weather_train = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_train.csv')
weather_test = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_test.csv')
build_meta = pd.read_csv('/kaggle/input/ashrae-energy-prediction/building_metadata.csv')

For a simple first model, we are going to merge the training sets to gather all covariables and make predictions on it with a linear regressor.

# <a id='2'>2. Combining Datasets</a>
We will merge everything into train and test dataframes.

In [ ]:
weather_test.timestamp = pd.to_datetime(weather_test.timestamp) 
weather_train.timestamp = pd.to_datetime(weather_train.timestamp) 

In [ ]:
#merge the building meta data and weather data into the train data
train_m = train.merge(build_meta, how='left', on = ['building_id'], validate='many_to_one') #merge the building meta data into the train data
train_m = train_m.merge(weather_train, how='left', on = ['site_id', 'timestamp'], validate='many_to_one')#add weather data to each time entry for each site ID

#merge the building meta data and weather data into the test data
test_m = test.merge(build_meta, how='left', on = ['building_id'], validate='many_to_one') #merge the building meta data into the train data
test_m = test_m.merge(weather_test, how='left', on = ['site_id', 'timestamp'], validate='many_to_one')#add weather data to each time entry for each site ID

train_m.head()

# <a id='3'>3. Missing Values</a>

The linear regressor does not work with NA values. To make it simple to understand, we decided to use the simple imputer with the 'most frequent' method which replaces NA values with the modal value.

In [ ]:
train_m.dropna()

In [ ]:
test_m.isna().describe()

# <a id='7'>7. Model Training</a>

Below we will tune the model parameters and get an idea of how well the model can perform on unseen data. We have done this by:

* Holding out the last 2.5 months of data for validation
* Holding out 10% of the buildings for validation
* Cross-validating time-series wise (for parameter tuning)


In [ ]:
%%time
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from lightgbm import LGBMRegressor
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, mean_squared_log_error

#defining a couple of functions for later use
def clip(x):
    return np.clip(x, a_min=0, a_max=None)
def rmse(y, y_pred):
    out = np.sqrt(mean_squared_error(clip(y), clip(y_pred)))
    return out

#prepare training data
X = train_m.dropna(subset=['meter_reading']) #drop all rows where the meter reading is not included
X = X.sort_values(by=['timestamp'], axis=0) #ensure X is sorted by timstamp for later timeseries cross-validation

builds = X['building_id'].unique()#array of building ids in the dataset
build_train, build_val = train_test_split(builds, test_size = 0.1, random_state=0)#hold out 10% of the buildings for validation

train = X.loc[(X['timestamp']<'2016-10-15') 
          & (X['building_id'].isin(build_train))] #we will train on only the first 80% of the year and 90% buildings
val_t = X.loc[(X['timestamp']>='2016-10-15') & (X['building_id'].isin(build_train))] #rest of the year and same buildings as above
val_b = X.loc[(X['building_id'].isin(build_val))] #full year and the rest of the buildings

y_train, y_val_t, y_val_b = train['meter_reading_rescaled'], val_t['meter_reading'], val_b['meter_reading'] #extracting the meter reading as our target variable
X_train, X_val_t, X_val_b = train.drop(['meter_reading', 'meter_reading_rescaled', 'timestamp'], axis=1), val_t.drop(['meter_reading', 'meter_reading_rescaled','timestamp'], axis=1), val_b.drop(['meter_reading','meter_reading_rescaled','timestamp'], axis=1)

del X, train, val_t, val_b #no longer needed - free up memory

# lgbm model
model = LGBMRegressor(
num_leaves = 600,
min_data_in_leaf = 50,
random_state = 0
)

#cross-validation for paramter tuning
# params = {
#     'num_leaves': [600],#add values to these lists to run a parmaeter optimization. These were found to be optimum.
#          }

# #define a rmse scorer for gridsearchcv
# rmse_scorer = make_scorer(rmse, greater_is_better=False)
# #split training data time series-wise for cross-validation
# tscv = TimeSeriesSplit(n_splits=3)
# #grid search
# #Note that the scores given are based on the rescaled meter readings, so are not a direct representation of model performance
# for model_name, grid in params.items():
#     searchCV = GridSearchCV(model, scoring=rmse_scorer, cv=tscv, param_grid=params)
#     print('GridSearchCV fitting...')
#     searchCV.fit(X_train, y_train)
#     scores = -1*searchCV.cv_results_['mean_test_score']
#     params = searchCV.cv_results_['params']
#     for i in range(0, len(scores)):
#       print(params[i], '->', scores[i])

#Evaluate combined model on the ramining validation data
print('Fitting...')
model.fit(X_train, y_train)

print('Time predictions...')
preds = clip(model.predict(X_val_t)) #make time predictions
preds_inv = scaler.inverse_transform(X_val_t, np.expm1(preds)) #convert back to original scale, remembering to invert the log transform
y_val_t = y_val_t.sort_index()
score = mean_absolute_error(preds_inv, y_val_t)
print('Mean absolute error - time prediction:', score)
RMSLE = np.sqrt(mean_squared_log_error(preds_inv, y_val_t))
print('RMSLE - time prediction:', RMSLE)

print('Building predictions...')
preds = clip(model.predict(X_val_b))
preds_inv = scaler.inverse_transform(X_val_b, np.expm1(preds))
y_val_b = y_val_b.sort_index()
score = mean_absolute_error(preds_inv, y_val_b)
print('Mean absolute error - new buildings:', score)
RMSLE = np.sqrt(mean_squared_log_error(preds_inv, y_val_b))
print('RMSLE - new buildings:', RMSLE)

Now that we have tuned the model parameters and have an idea of model performance. We will fit on the entire training dataset so we have as much information as possible for the final test set prediction.

In [ ]:
#prepare training data
train = train_m.dropna(subset=['meter_reading']) #drop all rows where the meter reading is not included

y_train = train['meter_reading_rescaled'] #extracting the meter reading as our target variable
X_train = train.drop(['meter_reading', 'meter_reading_rescaled', 'timestamp'], axis=1)

del train, train_m, X_val_t #no longer needed - free up memory
gc.collect()

#Fitting on all training data
print('Final Fitting...')
model.fit(X_train, y_train)

# <a id='8'>8. Model Predictions</a>

In [ ]:
#free up memory then load in the test data
del X_train, y_train
gc.collect()
X_test = pickle.load( open( "test_m.p", "rb" ) )

In [ ]:
#output the final predictions on the test data to a csv file
preds = np.empty(len(X_test))#we will predict in three steps to save memory
print('A')
preds[:int(len(X_test)/3)] = model.predict(X_test.drop(['row_id', 'timestamp'], axis=1).iloc[:int(len(X_test)/3)])
preds[int(len(X_test)/3):int(len(X_test)*2/3)] = model.predict(X_test.drop(['row_id', 'timestamp'], axis=1).iloc[int(len(X_test)/3):int(len(X_test)*2/3)])
preds[int(len(X_test)*2/3):] = model.predict(X_test.drop(['row_id', 'timestamp'], axis=1).iloc[int(len(X_test)*2/3):])
final_predictions = scaler.inverse_transform(X_test, np.expm1(preds))
X_test = X_test.sort_index()
output = pd.DataFrame({'row_id': X_test['row_id'], 'meter_reading': clip(final_predictions)})
output['meter_reading'] = output['meter_reading'].round(decimals=4)#to save space
output.to_csv('sub.csv', index_label = 'row_id', index=False)

As a final check, we can plot the model predictions with the existing data for a specific building and meter type. Everything during 2016 is real data while the rest is our final predictions.

In [ ]:
b_id = 400 #building id
m_id = 0 #meter id

train_m = pickle.load( open('train_m.p', 'rb'))
building_current = train_m.loc[(train_m['building_id']==b_id) & (train_m['meter']==m_id)]
building_forecast = X_test.loc[(X_test['building_id']==b_id) & (X_test['meter']==m_id)].merge(output, how='left', on = ['row_id'], validate='one_to_one')
building = pd.concat([building_current, building_forecast])

X_o = building.drop(['meter_reading', 'row_id', 'timestamp', 'site_id'], axis=1)
y_o = building['meter_reading']

mod_plot = pd.DataFrame(data={#'meter_reading (predicted)':building_forecast['meter_reading'],
                                    'meter_reading (actual and predicted)':y_o.values},
                                    index=building['timestamp'])
start_time = '2016-01-01'
end_time = '2019-01-01'
mod_plot = mod_plot.loc[(start_time<mod_plot.index)&(mod_plot.index<end_time)].resample('D').mean()
mod_plot.plot(rot=45)#plot each model vs target